In [3]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import mxnet as mx 
from data_ulti import toRecFile
from symbol import get_resnet_model
from symbol import YOLO_loss
from data_ulti import get_iterator

In [4]:
#convert raw image data to MXNET .rec file data
imgroot = "./DATA/"
annotation = np.load("./DATA/annotation_list.npy")[()]
imglist = annotation.keys()
sizet = 544
#rec_file = toRecFile(imgroot, imglist, annotation, sizet, (7,7,5), 32)

In [5]:
# get sym 
sym = get_resnet_model('pretrained_models/resnet-18', 0)
_, args_params, aux_params = mx.model.load_checkpoint('pretrained_models/resnet-18', 0)
logit = sym.get_internals()['logit_output']
#mx.viz.plot_network(sym)

In [6]:
# get some input 
train_data = mx.io.ImageRecordIter(path_imgrec='DATA_rec/cat_full.rec',
                                 data_shape=(3,224,224),
                                 label_width=7*7*5, 
                                 batch_size=1,)
val_data = mx.io.ImageRecordIter(path_imgrec='DATA_rec/cat_small.rec',
                                 data_shape=(3,224,224),
                                 label_width=7*7*5, 
                                 batch_size=1,)

In [7]:
# allocate memory
mod = mx.mod.Module(symbol=sym, context=mx.cpu(0))
# combine the two data
mod.bind(train_data.provide_data, train_data.provide_label)
mod.init_params(allow_missing=True, arg_params=args_params, aux_params=aux_params, 
                initializer=mx.init.Xavier(magnitude=2,rnd_type='gaussian',factor_type='in'))
out = mod.predict(eval_data=train_data, num_batch=1)
out[0]

<NDArray 1x7x7x5 @cpu(0)>